In [1]:
import pandas as pd
import numpy as np

### Wczytanie danych

In [2]:
df = pd.read_csv('australia.csv')
df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,17.9,35.2,0.0,12.0,12.3,48.0,6.0,20.0,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,0,0
1,18.4,28.9,0.0,14.8,13.0,37.0,19.0,19.0,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,0,0
2,19.4,37.6,0.0,10.8,10.6,46.0,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,0,0
3,21.9,38.4,0.0,11.4,12.2,31.0,6.0,6.0,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,0,0
4,24.2,41.0,0.0,11.2,8.4,35.0,17.0,13.0,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,0,0


In [3]:
df.shape

(56420, 18)

## Podział na zbiór treningowy i testowy

In [4]:
X = df.drop(columns=['RainTomorrow'])
y = df['RainTomorrow']

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Regresja logistyczna

In [32]:
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression

In [31]:
lr = LogisticRegression(max_iter=1000,                        
                        class_weight='balanced')
lr.fit(X_train, y_train)
y_pred2 = lr.predict(X_test)

print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.93      0.81      0.87      8856
           1       0.54      0.79      0.64      2428

    accuracy                           0.81     11284
   macro avg       0.73      0.80      0.75     11284
weighted avg       0.85      0.81      0.82     11284



1. Parametr `class_weight` nadaje każdej z klas wagę. Ustawienie tego parametru na `balanced` powoduje, że wagi są przyznawane klasom odwrotnie proporcjonalnie do częstości występowania danej klasy.
2. Parametr `max_iter` ogranicza liczbę iteracji, które zostaną wykonane.

## XGBClassifier

In [18]:
from xgboost import XGBClassifier

In [21]:
xg_boost = XGBClassifier(random_state=1,
                    learning_rate=0.1,
                    booster='gbtree',
                    max_depth=6,  
                    eval_metric='rmse',
                    use_label_encoder=False)

xg_boost.fit(X_train, y_train)
y_pred3 = xg_boost.predict(X_test)
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.88      0.95      0.92      8856
           1       0.75      0.53      0.62      2428

    accuracy                           0.86     11284
   macro avg       0.82      0.74      0.77     11284
weighted avg       0.85      0.86      0.85     11284



1. Parametr `learning_rate` odpowiada za szybkość uczenia.
2. `booster` określa strukturę modelu. Do wybory mamy: `gbtree`, `gblinear` oraz `dart`.
3. `max_depth` ustala maksymalną głębokość drzewa.
4. `eval_metric` - metryka używana do monitorowania uczenia modelu.
5. `use_label_encoder` przestarzały parametr, który zaleca się ustawić na `False`, a kodowanie zmiennych przeprowadzić z wykorzystaniem odpowiednich do tego narzędzi.

## KNeighborsClassifier

In [23]:
from sklearn.neighbors import KNeighborsClassifier

In [30]:
knn = KNeighborsClassifier(n_neighbors=3,
                          weights='distance',
                          p=1)

knn.fit(X_train, y_train)
y_pred4 = knn.predict(X_test)
print(classification_report(y_test, y_pred4))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90      8856
           1       0.66      0.54      0.59      2428

    accuracy                           0.84     11284
   macro avg       0.77      0.73      0.75     11284
weighted avg       0.83      0.84      0.83     11284



1. `n_neighbours` - liczba sąsiadów branych pod uwagę.
2. `weights` - wagi dla sąsiadów są nadawane odwrotnie propocjonalnie do odległości od analizowanego punktu. Czyli punkty bliżej mają większy wpływ.
3. `p` wykładnik potęgi w metryce Minkowskiego. Dla parametru p równego 1 mamy do czynienia z metryką taksówkową.

## Porównanie klasyfikatorów

Porównamy wszystkie trzy modele pod kątem accuracy, recall oraz f1-score.
1. W przypadku accuracy najlepiej wypada XGBClassifier.
2. W przypadku recall rózwnież najlepiej wypada XGBClassifier, bierzemy pod uwagę średnią ważoną z recall dla 0 i 1.
3. Również pod kątem f1-score XGBClassifier radzi sobie najlepiej. Tutaj również porównujemy po średniej ważonej z f1-score.

### Ranking
1. XGBClassifier
2. KNeighborsClassifier
3. LogisticRegression